In [8]:
import torch
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import os

In [9]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]

x_max = 1
y_max = 1
z_max = 1
sensor_points = np.array([[.5,.5,.5]])

ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [15]:
model = PINN([125,125,125])
model.set_location(sensor_points,[1,x_max,y_max,z_max],source_values=[1])
tfinal = 1.

In [16]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=4e-2)
from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.999)  # Decay LR by 5% every epoch

In [17]:
n=1000
sn = 250
best_loss = np.inf
sigma=.025
for epoch in range(30000):

    if epoch % 400 == 0:
        source_points,uv_points,source_values = model.source_points(sn,sigma)
        ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)
        xxx = torch.tensor(model.source_locs).repeat(len(collocation_points),1)
        ten = torch.sqrt(torch.sum(torch.square(collocation_points[:,1:] - xxx),axis=1))
        collocation_points = collocation_points[ten>sigma*3]
    # collocation_points = torch.cat([collocation_points,ic_col])
    # collocation_points.requires_grad=True

    uv = torch.ones(len(collocation_points),2)*.5
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,uv)
    loss_2,pde_2 = model.loss_function(source_points, uv_points, source_term = source_values)
    loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_points]),torch.concat([collocation_points,source_points]))

    # loss = loss_1 + loss_2*100
    # loss = loss_2*100
    loss = loss_1
    loss.backward()
    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()
    # scheduler.step()

    if epoch % 100 == 0:
        print(epoch, loss.item())
        print(loss_2.item(),loss_1.item())


0 0.061957292234851385
55.54736442142632 0.061957292234851385
100 0.01176789821109969
53.125640110471274 0.01176789821109969
200 0.011609026079105586
53.304862549995526 0.011609026079105586
300 0.01160901946087928
53.30384848156477 0.01160901946087928
400 0.005748416657572536
53.44462686168979 0.005748416657572536
500 0.00573846291503596
53.489206840959575 0.00573846291503596
600 0.005738461637726027
53.48930335652989 0.005738461637726027
700 0.005738460186652004
53.489305515672086 0.005738460186652004
800 0.02020038338160481
55.473183872581984 0.02020038338160481
900 0.020131445522964114
55.354111176987196 0.020131445522964114
1000 0.02013143253708212
55.353695113250126 0.02013143253708212
1100 0.020131416250867118
55.353705505559496 0.020131416250867118
1200 0.01996832558241541
53.60886707187412 0.01996832558241541
1300 0.019960759035187316
53.64812973803522 0.019960759035187316
1400 0.01996071015823761
53.64798860968014 0.01996071015823761
1500 0.019960622419769085
53.64805387072285

KeyboardInterrupt: 

In [6]:
print(x_max)

1


In [ ]:
model = torch.load('best_mod.m')

In [18]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = .5
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, z_max, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [19]:
# save as a GIF
import imageio
import os

images = []
source_loc = torch.tensor([[.5,.5,.5]])
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=True).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=.02)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[0,0], model.source_locs[0,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF



C:\Users\andyh\AppData\Local\Temp\ipykernel_32180\501775016.py:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [10]:
print(len(model.net.hidden))

4


In [11]:
print(source_points)

tensor([[0.7334, 0.4941, 0.5256, 0.4767],
        [0.4788, 0.5276, 0.4612, 0.4860],
        [0.3789, 0.5280, 0.4979, 0.5118],
        [0.6145, 0.5008, 0.4531, 0.4708],
        [0.1577, 0.4957, 0.4820, 0.5059],
        [0.7238, 0.5073, 0.5124, 0.4650],
        [0.6399, 0.4904, 0.5402, 0.5147],
        [0.9754, 0.5041, 0.4505, 0.5627],
        [0.3983, 0.5024, 0.4900, 0.5237],
        [0.7590, 0.4852, 0.4793, 0.5077],
        [0.6911, 0.4795, 0.4425, 0.5181],
        [0.4256, 0.5082, 0.4972, 0.4949],
        [0.9800, 0.5043, 0.5305, 0.4527],
        [0.6106, 0.4943, 0.5107, 0.5079],
        [0.1567, 0.4851, 0.5172, 0.5359],
        [0.6944, 0.4626, 0.4794, 0.5473],
        [0.3755, 0.4920, 0.5128, 0.5163],
        [0.2073, 0.4435, 0.5305, 0.4729],
        [0.9909, 0.5335, 0.4926, 0.4849],
        [0.8197, 0.5069, 0.4957, 0.4643],
        [0.6206, 0.4854, 0.5004, 0.4744],
        [0.6583, 0.4987, 0.4985, 0.4842],
        [0.2225, 0.4828, 0.5349, 0.4950],
        [0.5001, 0.5041, 0.5049, 0

In [12]:
print(source_values)

tensor([[0.3730],
        [0.1395],
        [0.4751],
        [0.0871],
        [0.7387],
        [0.3187],
        [0.2147],
        [0.0060],
        [0.5874],
        [0.5682],
        [0.0392],
        [0.9229],
        [0.0784],
        [0.8450],
        [0.2357],
        [0.0389],
        [0.6746],
        [0.0206],
        [0.3241],
        [0.3414],
        [0.4984],
        [0.8171],
        [0.2920],
        [0.9215],
        [0.0090],
        [0.0254],
        [0.3904],
        [0.6471],
        [0.3525],
        [0.5598],
        [0.1894],
        [0.0025],
        [0.1525],
        [0.3662],
        [0.0573],
        [0.4913],
        [0.8808],
        [0.6022],
        [0.1590],
        [0.4709],
        [0.2174],
        [0.1879],
        [0.4791],
        [0.0406],
        [0.0379],
        [0.5956],
        [0.1011],
        [0.2587],
        [0.5458],
        [0.5405],
        [0.5782],
        [0.6503],
        [0.6471],
        [0.7722],
        [0.3796],
        [0